In [1]:
from keras.datasets import cifar10
from keras.models import load_model
from keras.utils.np_utils import to_categorical
import numpy as np
from six.moves import cPickle as pickle
import os.path
from IPython import display
import matplotlib.pyplot as plt

% matplotlib inline
% config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [2]:
def prepData(image_class):
    # Fetch data
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # Convert uint8 pixel values to float32 in the range [0, 1] (for sigmoid)
    X_test = X_test.astype('float32')
    X_test /= 255

    # Split dataset
    X_new = np.ndarray(shape=(X_test.shape[0] / 10, X_test.shape[1], 
                        X_test.shape[2], X_test.shape[3]), dtype=np.float32)
    X_other = np.ndarray(shape=(9 * X_test.shape[0] / 10, X_test.shape[1], 
                        X_test.shape[2], X_test.shape[3]), dtype=np.float32)

    sampleNumber = 0
    otherNumber = 0
    for X, y in zip(X_test, y_test):
        if y[0] == image_class:
            X_new[sampleNumber] = X
            sampleNumber += 1
        else:
            X_other[otherNumber] = X
            otherNumber += 1
    X_test = X_new[:, :, :, :]

    return X_test, X_other

In [13]:
for epoch in [50, 100, 150, 200, 250, 300]:
    avg_accuracy = 0
    for image_class in range(10):
        X_test, X_other = prepData(image_class)
        y_test = np.zeros(shape=(X_test.shape[0], 3))
        y_other = np.zeros(shape=(X_other.shape[0], 3))
        y_test[:, 0] = 1 # Real class images
        y_other[:, 2] = 1 # Real other images
        
        other_indices = np.random.randint(0, X_other.shape[0], X_test.shape[0])
        X_test = np.concatenate((X_test, X_other[other_indices, :, :, :]))
        y_test = np.concatenate((y_test, y_other[other_indices, :]))

        discriminator = load_model("networks/disc" + str(image_class) + "-" + str(epoch) + ".h5")

        preds = discriminator.predict(X_test)

        accuracy = 0
        real_predictions = 0
        gen_predictions = 0
        for pred_index in range(len(preds)):
            if y_test[pred_index, 0] == 1:
                if preds[pred_index, 0] + preds[pred_index, 1] >= preds[pred_index, 2]:
                    accuracy += 1
                    if preds[pred_index, 0] > preds[pred_index, 1]:
                        real_predictions += 1
                    else:
                        gen_predictions += 1
            elif y_test[pred_index, 2] == 1:
                if preds[pred_index, 0] + preds[pred_index, 1] <= preds[pred_index, 2]:
                    accuracy += 1
            else:
                print "y_test should not include generated images"

        accuracy /= 1. * len(preds)
        real_gen_ratio = 1. * real_predictions / (real_predictions + gen_predictions)
        
        msg = "Epoch: {:d}, Image class: {:d}".format(epoch, image_class)
        msg += ", Accuracy: {:.4f}, Real Pred. Ratio: {:.4f}".format(accuracy, real_gen_ratio)
        print msg
        
        avg_accuracy += accuracy
    print "\nEpoch: {:d} Average Accuracy: {:.4f}\n".format(epoch, avg_accuracy / 10)

Epoch: 50, Image class: 0, Accuracy: 0.9015, Real Pred. Ratio: 0.5686
Epoch: 50, Image class: 1, Accuracy: 0.9445, Real Pred. Ratio: 0.7238
Epoch: 50, Image class: 2, Accuracy: 0.7910, Real Pred. Ratio: 0.3925
Epoch: 50, Image class: 3, Accuracy: 0.8080, Real Pred. Ratio: 0.7914
Epoch: 50, Image class: 4, Accuracy: 0.8815, Real Pred. Ratio: 0.5559
Epoch: 50, Image class: 5, Accuracy: 0.8360, Real Pred. Ratio: 0.6908
Epoch: 50, Image class: 6, Accuracy: 0.8350, Real Pred. Ratio: 0.2953
Epoch: 50, Image class: 7, Accuracy: 0.9185, Real Pred. Ratio: 0.9235
Epoch: 50, Image class: 8, Accuracy: 0.9305, Real Pred. Ratio: 0.1844
Epoch: 50, Image class: 9, Accuracy: 0.9155, Real Pred. Ratio: 0.8980

Epoch: 50 Average Accuracy: 0.8762

Epoch: 100, Image class: 0, Accuracy: 0.9245, Real Pred. Ratio: 0.6729
Epoch: 100, Image class: 1, Accuracy: 0.9665, Real Pred. Ratio: 0.9017
Epoch: 100, Image class: 2, Accuracy: 0.8505, Real Pred. Ratio: 0.6957
Epoch: 100, Image class: 3, Accuracy: 0.8400, Real

In [23]:
# Fetch data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert uint8 pixel values to float32 in the range [0, 1] (for sigmoid)
X_test = X_test.astype('float32')
X_test /= 255

for epoch in [50, 100, 150, 200, 250, 300]:
    all_class_preds = np.ndarray(shape=(X_test.shape[0], 10), dtype=np.float32)
    for image_class in range(10):
        discriminator = load_model("networks/disc" + str(image_class) + "-" + str(epoch) + ".h5")
        preds = discriminator.predict(X_test)
        class_preds = preds[:, 0] + preds[:, 1]
        all_class_preds[:, image_class] = class_preds
    
    # Calculate joint accuracy
    all_class_preds_index = np.argmax(all_class_preds, axis=1)
    accuracy = 0
    for y, p in zip(y_test, all_class_preds_index):
        if y == p:
            accuracy += 1
    accuracy /= 1. * len(y_test)
    print "Epoch {:d} joint accuracy: {:.4f}".format(epoch, accuracy)

Epoch 50 joint accuracy: 0.6975
Epoch 100 joint accuracy: 0.7952
Epoch 150 joint accuracy: 0.8188
Epoch 200 joint accuracy: 0.8265
Epoch 250 joint accuracy: 0.8313
Epoch 300 joint accuracy: 0.8316
